![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Scraping  
  
Pour cette deuxième partie, nous allons récupérer des données à partir de la méthode de scraping.  
Cette méthode permet de récupérer directement l'information sur un site internet "Booking".  
  
**Objectifs**  
Nous avons un DataFrame composé de villes, classées selon le taux de précipitation.  
Après avoir choisi une ville, l'objectif est d'aller rechercher:  
    * Le nom de l'hotel  
    * L'url vers sa page Booking  
    * Le scrore donné par les utilisateurs  
    * Description textuelle de l'hôtel

#### Importation des librairies

In [1]:
import pandas as pd
import os
import logging

import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
df = pd.read_csv('destinations.csv') # Chargement du DataFrame obtenue dans la première partie
df.head()

,id_ville,Ville,latitude,longitude,main_weather,expected_rain
0,7,Lille,50.636565,3.063528,Clouds,3.6870
1,5,Paris,48.856697,2.351462,Clear,3.7847
2,6,Amiens,49.894171,2.295695,Clouds,4.5048
3,0,Mont Saint Michel,48.635523,-1.510257,Rain,5.3364
4,4,Rouen,49.440459,1.093966,Clouds,6.4300


# Hôtel 1

In [3]:
class BookingSpider(scrapy.Spider):

    # Le nom du spider
    name = "booking_info" # si j'utilise spacy

    # Url où le spider va commencer
    start_urls = [
            'https://www.booking.com/index.fr.html',
        ]

    def parse(self, response): 
                return scrapy.FormRequest.from_response(
                response,
                formdata={'ss': '{}'.format(destination_ville)},
                callback=self.after_search
                )
        # response est un objet qui représente une réponse http, transmit aux spiders
        
    def after_search(self, response):
        """ 
        la fonction permet de scrapper booking (grâce aux balises html) et de retourner les valeurs suivantes :
        - nom de l'hotel
        - url
        - score de l'hotel 
        - description de l'hotel 
        """
        hotel_name = response.css('span.sr-hotel__name')
        hotel_url = response.css('h3 a')
        hotel_note = response.css('div.bui-review-score__badge')
        hotel_desc = response.css('div.hotel_desc')
       
        for url, name, note, desc in zip(hotel_url,hotel_name,hotel_note,hotel_desc):
            yield {
                'hotel_name': name.css('::text').get(),
                'hotel_url': url.attrib['href'],
                'hotel_note': note.css('::text').get(),
                'hotel_description': desc.css('::text').get()
            }
            
            

In [4]:
destination_ville = input('Quelle ville souhaitez vous visiter ?') # On demande le nom de la ville

filename = "hotel_{}.json".format(destination_ville)  # On créer un nouveau ficher json, qui contiendra l'ensemble des informations scrappées 
if os.path.exists(filename):
    os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    }
})

process.crawl(BookingSpider) # 
process.start()

Quelle ville souhaitez vous visiter ? Paris


2021-09-13 14:49:30 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-09-13 14:49:30 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.10 (default, May 19 2021, 13:12:57) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.19042-SP0
2021-09-13 14:49:30 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-09-13 14:49:30 [scrapy.extensions.telnet] INFO: Telnet Password: 954623db1d949139
2021-09-13 14:49:30 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-09-13 14:49:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth